In [59]:
import pandas as pd
import numpy as np
import os
import itertools
from openpyxl import load_workbook
from IPython.display import display

In [60]:
# File to be reading
file = 'ceník Q10.xlsx'
file_path = f"podklady/{file}"

# Sheet Name
sheet_name = 'ZASKLENÝ RÁM'

# Which row should be skipped
skiprows = 30

# Till wich row shold program read rows
nrows = 26

# Which columns shoud program read
usecols = 'A:W'
#usecols = 'J:P'

# barva
#barva = "weiss"
barva = "dekor einseitig"
#barva = "dekor beidseitig"

# nová tabulka - jednotlivé položky
material = ["?"]
breite = []
hohe = []
preis = []
preismatrix = [f"Q10 PREMIUM {barva}"]
product_key = ["Fenster FIX (Glas im Rahmen)"]

# Cesta k novému souboru
path = f"prevedene_tabulky/upraveno_{file}"
#sheet = f"{product_key[0]} {preismatrix[0][-5:]}"
sheet = f"{preismatrix[0][0:3]} {product_key[0]} {barva}"

# # Finální tabulka
new_data = []

In [61]:
kec = "blah"
data = [f"some text {kec}"]
#print(data[0])

In [62]:
# Read data from file and specific sheet. Read data from a specific range
pd.set_option("display.max_row", None)
df = pd.read_excel(file_path,sheet_name=sheet_name, header = None, skiprows = skiprows, nrows=nrows,  usecols= usecols)
df.columns = range(df.columns.size)
#display(df)

In [63]:
df = df.replace([np.nan, 'V/Š'], [0, 0], regex=True)
df = df.astype('int')
#display(df)

In [64]:
# Počet řádku v tabulce
table_row = len(df)-1
#print(table_row)

In [65]:
# Počet sloupců v tabulce
table_column = len(df.columns)-1
#print(table_column)

In [66]:
# Hohe - Výška
for index, row in df.iterrows():
    hohe.append(row[0])
hohe.pop(0)
hohe = [el for el in hohe for i in range(table_column)]
#print(hohe)

In [67]:
# Breite - Šířka
for column in df:
    breite.append(df[column][0])
breite.pop(0)

breite = breite*table_row
#print(breite)

In [68]:
# Ceny v listu
df_list = df.values.tolist()
del df_list[0]

for i in df_list:
    del i[0]

preis = list(itertools.chain.from_iterable(df_list))

In [69]:
lst = [material *len(preis),breite, hohe, preis, preismatrix*len(preis), product_key*len(preis)]

# Vytváření tabulky z listů uvedených v lst
table = pd.DataFrame(lst)

# Otočení tabulky
final_table = table.transpose()

# Pojmenování jednotlivých sloupců
final_table.columns = ["Material", "Breite", "Hohe", "Preis", "Preismatrix", "Product Key"]

# Odstranění nulových hodnot podle sloupce "Preis" 
final_table = final_table[final_table.Preis != 0]
#final_table

In [70]:
if os.path.exists(path):
    book = load_workbook(path)
    writer = pd.ExcelWriter(path, engine = 'openpyxl')
    writer.book = book
    final_table.to_excel(writer, sheet_name = sheet, index=False)
    writer.close()
else:
    final_table.to_excel(path, sheet_name = sheet, index=False)

/home/honza/Programs/spyder/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
